## **Этот ноутбук**: использует модель qwen 3.8b генерирует натуральные запросы, что могли написать реальные пользователи при поиске фильмов. Код построен так, что запросы разные и задевают разные аспекты фильмов.

# Объединяем получившиеся описания в один датасет

In [ ]:
# df = pd.read_parquet("/content/plot_prompt_desc (1).parquet")
# df_2 = pd.read_parquet("/content/plot_prompt_desc.parquet")


# import pandas as pd

# # допустим, df1 и df2 имеют одинаковые колонки
# merged = pd.concat([df, df_2], ignore_index=True)
# merged.to_parquet()

In [ ]:
# merged = merged.drop_duplicates().shape
# merged = merged[merged['desc']!='']
# print(merged['plot'].nunique())

# merged.to_parquet("/content/plot_prompt_desc.parquet")

In [ ]:
# df = pd.read_parquet("/content/plot_prompt_desc.parquet")

In [ ]:
# import string

# ASPECT_ORDER = [
#     "genre","subgenre","tone","setting_place","setting_time",
#     "protagonist_archetype","antagonist_type","relationship_dynamics",
#     "core_goal","inciting_incident","themes","scale",
#     "narrative_structure","pacing","ending_tone"
# ]

# def parse_one_output(text: str) -> dict[str, list[str]]:
#     # чистка пунктуации (кроме "_")
#     punct = string.punctuation.replace('_', '')

#     def drop_na(lst):
#         return [i for i in lst if i]

#     def clean_(txt: str) -> str:
#         return txt.translate(str.maketrans('', '', punct)).lower().strip()

#     lines = [clean_(ln) for ln in text.split("\n") if ln.strip()]
#     dict_aspect = {asp: ["not_specified"]*3 for asp in ASPECT_ORDER}

#     for line in lines:
#         parts = drop_na(line.split())
#         if not parts:
#             continue
#         asp = parts[0]
#         if asp in dict_aspect:
#             vals = parts[1:4]
#             # паддинг до 3
#             while len(vals) < 3:
#                 vals.append("not_specified")
#             dict_aspect[asp] = vals[:3]
#     return dict_aspect

In [ ]:
# mini_df = {asp: [] for asp in ASPECT_ORDER}
# for i in range(len(df)):
#   dct = parse_one_output(df.iloc[i,2])
#   for asp in ASPECT_ORDER:
#     asp_list = dct[asp]
#     asp_list = [c for c in asp_list if c!="not_specified"]
#     if asp_list==[]:
#       asp_values=''
#     else:
#       asp_values=', '.join(asp_list)
#     mini_df[asp].append(asp_values)


In [ ]:

# for asp in ASPECT_ORDER:
#   df[asp] = mini_df[asp]

In [ ]:
# all = len(df)
# percent_missed = pd.DataFrame({'aspect': [], 'perc_missed': []})
# percent_missed['aspect'] = ASPECT_ORDER
# lst = []
# for asp in ASPECT_ORDER:
#   pm = round(len(df[df[asp]==''])/all*100, 2)
#   lst.append(pm)
#   print(f'Missed data for aspect "{asp}" are ', pm, '%', sep='')

# percent_missed['perc_missed'] = lst


# Answer:

# all = len(df)
# percent_missed = pd.DataFrame({'aspect': [], 'perc_missed': []})
# percent_missed['aspect'] = ASPECT_ORDER
# lst = []
# for asp in ASPECT_ORDER:
#   pm = round(len(df[df[asp]==''])/all*100, 2)
#   lst.append(pm)
#   print(f'Missed data for aspect "{asp}" are ', pm, '%', sep='')

# percent_missed['perc_missed'] = lst


# Answer:
# Missed data for aspect "genre" are 0.23%

# Missed data for aspect "subgenre" are 41.1%

# Missed data for aspect "tone" are 1.06%

# Missed data for aspect "setting_place" are 2.88%

# Missed data for aspect "setting_time" are 6.5%

# Missed data for aspect "protagonist_archetype" are 7.19%

# Missed data for aspect "antagonist_type" are 17.64%

# Missed data for aspect "relationship_dynamics" are 32.77%

# Missed data for aspect "core_goal" are 7.91%

# Missed data for aspect "inciting_incident" are 20.04%

# Missed data for aspect "themes" are 36.43%

# Missed data for aspect "scale" are 70.69%

# Missed data for aspect "narrative_structure" are 76.37%

# Missed data for aspect "pacing" are 89.37%

# Missed data for aspect "ending_tone" are 98.45%


In [ ]:
# percent_missed = percent_missed.sort_values(by='perc_missed')

In [ ]:
# ok_aspect = percent_missed[percent_missed['perc_missed']<45]['aspect'].to_list()

In [ ]:
# # оставляем следующие аспекты:
# ok_aspect


# Answer:
# ['genre',
# , 'tone',
# , 'setting_place',
# , 'setting_time',
# , 'protagonist_archetype',
# , 'core_goal',
# , 'antagonist_type',
# , 'inciting_incident',
# , 'relationship_dynamics',
# , 'themes',
# , 'subgenre']

In [ ]:
# nead_col = df.columns[:3].to_list()+ok_aspect
# ok_df = df[nead_col]

In [ ]:
# ok_df.shape

In [ ]:
# count_nones = {i: 0 for i in range(len(ok_aspect))}
# col_count_nones = []
# for i in range(len(ok_df)):
#   c = 0
#   for asp in ok_aspect:
#     if ok_df.iloc[i][asp]=='': c+=1
#   count_nones[c]+=1
#   col_count_nones.append(c)

In [ ]:
# ok_df['count_nones'] = col_count_nones

# Выбираем только строки где заполнены 8 и более аспектов.
## Зачем? - группа аспектов делится на под группы по 2-3 штуки. Если их будет, например 7, то в одной группе будет 1 аспект, а не 2.

In [ ]:
# ok_df = ok_df[ok_df['count_nones']<=3]

In [ ]:
# ok_df.shape

# Создаём промпты.
## Логика:
### - 6 уникальных сабсетов, перескаются не более чем на 2 аспекта
### - 5 уникальных стилей с примерами для модели
### - 2 уник. стиля на сабсет = 12 уникальных промптов

In [ ]:
import pandas as pd

In [ ]:
ok_df = pd.read_parquet("/kaggle/input/prompts-for-q/ok_df.parquet")

In [ ]:
import random
import pandas as pd
from itertools import combinations

# --- Конфиг ---

ASPECT_COLS = [
    "genre","tone","setting_place","setting_time","protagonist_archetype",
    "core_goal","antagonist_type","inciting_incident","relationship_dynamics",
    "themes","subgenre"
]

# 5 стилей × 3 примера — как ты дал, НО мы их превратим в строки "список (номер): ...\nответ: ..."
STYLE_EXAMPLES_RAW = {
    "genre_theme": [
        ("genre: comedy; subgenre: dark comedy; themes: dysfunctional family; setting_place: suburb",
         "dark suburban comedy about a dysfunctional family"),
        ("genre: thriller; themes: revenge; subgenre: psychological; tone: tense",
         "tense psychological thriller about revenge"),
        ("genre: sci-fi; themes: identity; setting_place: space station; tone: contemplative",
         "contemplative sci-fi set on a space station about identity"),
    ],
    "mini_plot": [
        ("genre: horror; inciting_incident: strange noises at night; setting_place: remote cabin; antagonist_type: supernatural entity",
         "in a remote cabin, strange nightly noises hint at a supernatural presence"),
        ("genre: crime; inciting_incident: unexplained murder; protagonist_archetype: small-town cop; setting_place: rural town",
         "a small-town cop investigates an unexplained murder in a rural town"),
        ("genre: drama; core_goal: reunite family; relationship_dynamics: estranged siblings; setting_time: winter",
         "estranged siblings trying to reunite their family during winter"),
    ],
    "character": [
        ("protagonist_archetype: outcast detective; themes: corruption; setting_place: coastal town; tone: bleak",
         "bleak coastal-town story of an outcast detective uncovering corruption"),
        ("protagonist_archetype: young witch; themes: identity; setting_time: medieval; setting_place: village",
         "a young witch in a medieval village struggling with identity"),
        ("protagonist_archetype: single mother; core_goal: rebuild life; genre: drama; relationship_dynamics: parent-child tension",
         "drama about a single mother rebuilding life amid parent-child tension"),
    ],
    "atmosphere": [
        ("genre: mystery; tone: tense; setting_place: foggy forest; subgenre: psychological",
         "tense psychological mystery in a foggy forest"),
        ("genre: romance; tone: bittersweet; setting_time: winter; setting_place: small city",
         "bittersweet winter romance in a small city"),
        ("genre: horror; tone: dreamlike; setting_place: abandoned hospital; subgenre: surreal",
         "dreamlike surreal horror in an abandoned hospital"),
    ],
    "effect": [
        ("genre: drama; themes: family loss; tone: tragic; relationship_dynamics: parent-child grief",
         "heartbreaking family drama about parent-child grief and loss"),
        ("genre: thriller; inciting_incident: sudden betrayal; tone: shocking; antagonist_type: criminal syndicate",
         "shocking thriller about betrayal and a ruthless criminal syndicate"),
        ("genre: horror; tone: dread-heavy; subgenre: no-gore; themes: isolation",
         "dread-heavy no-gore horror about isolation"),
    ],
}

PROMPT_TMPL = """You will receive a compact movie description by facets.

TASK:
Write ONE short, natural mini-description that a user could type in a streaming search. It must read as a single coherent sentence, not a tag list. Return ONE line with the mini-description only. If you output anything else, it will be discarded.

CONSTRAINTS:
- 5–20 words
- No movie titles, no quotes, no hashtags, no question marks
- Use only information implied by the given facets (no invention)
- Keep it fluent and human


EXAMPLES (the list of facets may not match yours):
<<examples_block>>


Now it’s your turn to answer. I will give you FACETS, and you should create a mini-description. Do not copy the facets I provide. Follow the spirit of the examples, but create an original result.
Your FACETS:
<<facets_subset>>

Your OUTPUT:"""

# --- Утилиты ---


def collect_nonempty_facets(row) -> dict:
    d = {}
    for col in ASPECT_COLS:
        if col in row and row[col]!='':
            d[col] = str(row[col]).strip()
    return d

# --- Утилиты ---

def examples_block_for_style(style: str) -> str:
    """Форматируем 3 примера как список строк 'список (i): ...\\nответ: ...' """
    pairs = STYLE_EXAMPLES_RAW[style]
    lines = []
    for i, (facets, out) in enumerate(pairs, 1):
        lines.append(f"Example {i}: /nInput facets: {facets}\nOUTPUT: {out}")
    return "\n\n".join(lines)


def collect_nonempty_facets(row) -> dict:
    return {col: str(row[col]).strip() for col in ASPECT_COLS if col in row and row[col]!=''}

def random_4_groups(names):
    """
    Делим список из 8..11 аспектов на 4 непересекающихся группы 2–3.
    """
    names = list(names)
    random.shuffle(names)
    sizes = [2,2,2,2]
    extra = len(names) - 8  # 0..3
    i = 0
    while extra > 0:
        if sizes[i] < 3:
            sizes[i] += 1
            extra -= 1
        i = (i + 1) % 4
    groups, idx = [], 0
    for sz in sizes:
        groups.append(names[idx:idx+sz])
        idx += sz
    return groups  # 4 группы по 2–3 ключа

def make_6_subsets_from_groups(groups):
    """Все пары групп (C(4,2)=6) → 6 сабсетов."""
    subs = []
    for i, j in combinations(range(4), 2):
        subs.append(tuple(sorted(groups[i] + groups[j])))
    # уникализация на всякий случай
    uniq, seen = [], set()
    for s in subs:
        if s not in seen:
            seen.add(s)
            uniq.append(s)
    return uniq  # должно быть 6

def build_facets_subset_text(subset_keys, facets_dict) -> str:
    """Формируем блок FACETS: построчно 'ключ: значение' (без тире, как в твоём шаблоне)."""
    return "\n".join(f"{k}: {facets_dict[k]}" for k in subset_keys if k in facets_dict)

def assign_two_styles_per_subset(subsets, all_styles=("genre_theme","mini_plot","character","atmosphere","effect")):
    """
    Для каждого сабсета выбираем 2 разных стиля (из 5), порядок перемешиваем.
    Возвращает список кортежей: (subset_keys_tuple, style)
    Итого 12 элементов.
    """
    pairs = []
    for sub in subsets:
        two = random.sample(list(all_styles), 2)
        random.shuffle(two)
        for st in two:
            pairs.append((sub, st))
    return pairs  # len == 12

def build_prompts_for_row(row) -> list:
    """
    На вход: одна строка исходного DF (есть 'desc' и 11 аспектов).
    На выход: список из 12 словарей {"desc": ..., "prompt": ...}.
    """
    facets = collect_nonempty_facets(row)                   # 8..11 заполненных аспектов
    if not (8 <= len(facets) <= 11):
        return []
    groups = random_4_groups(list(facets.keys()))          # 4 группы по 2–3
    subsets = make_6_subsets_from_groups(groups)           # 6 сабсетов (пары групп)
    subset_style_pairs = assign_two_styles_per_subset(subsets)  # 12 пар (subset, style)

    out_rows = []
    for subset_keys, style in subset_style_pairs:
        examples_block = examples_block_for_style(style)   # ОДИН стиль на промт
        facets_subset_txt = build_facets_subset_text(subset_keys, facets)

        prompt = PROMPT_TMPL.replace("<<examples_block>>", examples_block)\
                             .replace("<<facets_subset>>", facets_subset_txt)

        out_rows.append({"desc": row.get("desc",""), "prompt": prompt})
    return out_rows

def build_desc_prompt_df(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Пробегаем по всему входному DF и собираем выходной DF с колонками: desc, prompt.
    По каждой строке → 12 промтов.
    """
    rows = []
    for _, r in df_in.iterrows():
        rows.extend(build_prompts_for_row(r))
    return pd.DataFrame(rows, columns=["desc","prompt"])

df = build_desc_prompt_df(ok_df)

In [ ]:
df['prompt'][0]

'You will receive a compact movie description by facets.\n\nTASK:\nWrite ONE short, natural mini-description that a user could type in a streaming search. It must read as a single coherent sentence, not a tag list. Return ONE line with the mini-description only. If you output anything else, it will be discarded.\n\nCONSTRAINTS:\n- 5–20 words\n- No movie titles, no quotes, no hashtags, no question marks\n- Use only information implied by the given facets (no invention)\n- Keep it fluent and human\n\n\nEXAMPLES (the list of facets may not match yours):\nExample 1: /nInput facets: genre: horror; inciting_incident: strange noises at night; setting_place: remote cabin; antagonist_type: supernatural entity\nOUTPUT: in a remote cabin, strange nightly noises hint at a supernatural presence\n\nExample 2: /nInput facets: genre: crime; inciting_incident: unexplained murder; protagonist_archetype: small-town cop; setting_place: rural town\nOUTPUT: a small-town cop investigates an unexplained murde

## Датасет добавлен на kaggle

# определяем max_new_tokens

In [ ]:

# # узнаваем восколько в текстах количество токенов больше количество слов. используем как коэфицент.

# def count_words(text):
#   return len(text.split())

# def count_tokens(text):
#   return len(tokenizer(text, return_tensors="pt", padding=True, truncation=True)['input_ids'][0])

# prompts_to_q['prompt_in_words'] = prompts_to_q['prompt'].apply(count_words)
# prompts_to_q['prompt_in_tokens'] = prompts_to_q['prompt'].apply(count_tokens)

# import matplotlib.pyplot as plt

# plt.plot(prompts_to_q['prompt_in_words'], label='prompt_in_words')
# plt.plot(prompts_to_q['prompt_in_tokens'], label='prompt_in_tokens')

# plt.legend()
# plt.show()

# # 20 - максимальная длинная запроса по промту
# prompts_to_q['nc'] = prompts_to_q['prompt_in_tokens']/prompts_to_q['prompt_in_words']
# print(max(prompts_to_q['nc'])*20)

# # результат: 38 ~ 40

In [ ]:
# # средняя длина промта
# prompts_to_q['prompt_in_tokens'].describe()

# Финал. Этап генерации запросов.

In [ ]:

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
import os

# пример: если секрет называется "MY_API_KEY"

from huggingface_hub import login
import os


login(token=hf_token)

import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import string


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os, torch

import pandas as pd

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True,garbage_collection_threshold:0.6"
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.enable_flash_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_math_sdp(False)

name = "microsoft/phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    name,
    torch_dtype=torch.float16,
    device_map="auto",                       # model-parallel по всем GPU
    low_cpu_mem_usage=True
)
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=32064, 

In [ ]:
import pandas as pd

df['queries'] = ''
df = df.sort_values(by="prompt", key=lambda col: col.str.len())
df = df.iloc[36960:]


max_new_tokens = 25

saved_per_batch = 8
batch_size = 42
n = 0
for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    prompts = batch['prompt'].tolist()

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to('cuda')

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            # temperature=0.0
        )

    decoded = []
    input_len = inputs["input_ids"].shape[1]
    for output in outputs:
        text = tokenizer.decode(output[input_len:], skip_special_tokens=True)
        text = text.split("\n\n\n")[0].replace('\n', '')
        decoded.append(text)

    # сохраняем обратно
    df.loc[batch.index, 'queries'] = decoded

    # периодическое сохранение
    if (i // batch_size + 1) % saved_per_batch == 0:
        df.to_parquet("queries_and_desc.parquet", index=False)
df.to_parquet("queries_and_desc.parquet", index=False)